<a href="https://colab.research.google.com/github/ogutiann/6COSC019C-Cyber-Security/blob/main/POTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from deap import algorithms, base, creator, tools
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import gc
from tqdm import tqdm
import pygad
import networkx as nx
from matplotlib.colors import LinearSegmentedColormap

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Configuration for large-scale deployment
N_WORKERS = 5000
N_TASKS = 10000
CAPACITY = 3  # Max tasks per worker
GPU_ACCELERATED = torch.cuda.is_available()

if GPU_ACCELERATED:
    device = torch.device("cuda")
    print(f"🚀 Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU - GPU recommended for large-scale simulation")

######################
### 1. System Model ##
######################
class Worker:
    def __init__(self, id):
        self.id = id
        self.location = np.array([np.random.uniform(0,100), np.random.uniform(0,100)])
        self.skill = np.array([np.random.uniform(2,5), np.random.uniform(2,5)])
        self.speed = np.random.uniform(0.5, 2)
        self.weights = np.random.dirichlet(np.ones(3))
        self.reputation = np.random.uniform(0.7, 0.95)
        self.utility_history = []
        self.assigned_tasks = []

    def travel_time(self, task_location):
        distance = np.linalg.norm(self.location - task_location)
        traffic_noise = np.random.laplace(0, 0.1)
        return max(0.1, distance / self.speed + traffic_noise)

class Task:
    def __init__(self, id):
        self.id = id
        self.location = np.array([np.random.uniform(0,100), np.random.uniform(0,100)])
        self.reward = np.random.uniform(5,20)
        self.deadline = np.random.randint(10,60)
        self.difficulty = np.random.uniform(1,10)
        self.required_skill = np.array([np.random.uniform(1,4), np.random.uniform(1,4)])

################################
### 2. Deep-Quality Reputation ##
################################
class QualityVAE(nn.Module):
    def __init__(self, input_dim=3, latent_dim=8):
        super(QualityVAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.mu = nn.Linear(16, latent_dim)
        self.logvar = nn.Linear(16, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.mu(h), self.logvar(h)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + 0.5 * KLD

def compute_quality(features, vae_model):
    with torch.no_grad():
        recon, _, _ = vae_model(features)
        error = torch.norm(features - recon, dim=1)
    return 1 - error

################################################
### 3. Federated RL for Preference Optimization #
################################################
class LSTMPolicy(nn.Module):
    def __init__(self, input_size=5, hidden_size=32, output_size=3):
        super(LSTMPolicy, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])
        return torch.softmax(x, dim=-1)

def federated_average(models):
    global_dict = {}
    for key in models[0].state_dict().keys():
        global_dict[key] = torch.stack([m.state_dict()[key] for m in models]).mean(0)
    return global_dict

def train_worker_rl(worker, tasks, epochs=20, batch_size=64):
    model = LSTMPolicy().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.MSELoss()

    # Simulate historical data
    n_samples = 1000
    states = torch.randn(n_samples, 1, 5, device=device)  # [unmatched_ratio, rep, avg_util, travel_time, deadline_pressure]
    actions = torch.randint(0, 3, (n_samples,), device=device)
    rewards = torch.rand(n_samples, device=device)

    # Training loop
    for epoch in range(epochs):
        permutation = torch.randperm(n_samples)
        for i in range(0, n_samples, batch_size):
            indices = permutation[i:i+batch_size]
            batch_states = states[indices]
            batch_actions = actions[indices]
            batch_rewards = rewards[indices]

            # Forward pass
            action_probs = model(batch_states)
            selected_probs = action_probs[torch.arange(len(batch_actions)), batch_actions]

            # Policy gradient loss
            loss = -torch.log(selected_probs) * batch_rewards
            loss = loss.mean()

            # Optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model

#############################################
### 4. Matching Algorithms Implementations ##
#############################################
def worker_utility(worker, task):
    tau = worker.travel_time(task.location)
    skill_gap = np.mean(worker.skill - task.required_skill)
    return (worker.weights[0] * (1/tau) +
            worker.weights[1] * task.reward +
            worker.weights[2] * skill_gap +
            0.3 * worker.reputation)

def task_utility(task, worker):
    if np.any(worker.skill < task.required_skill):
        return 0
    tau = worker.travel_time(task.location)
    return (0.7 * (1/tau) + 0.3 * worker.reputation)

# 4.1 RAGS (Our Proposed)
def rags_matching(workers, tasks, capacity=CAPACITY):
    # Precompute utilities
    w_utils = np.zeros((len(workers), len(tasks))
    t_utils = np.zeros((len(tasks), len(workers)))

    for i, worker in enumerate(workers):
        for j, task in enumerate(tasks):
            w_utils[i, j] = worker_utility(worker, task)
            t_utils[j, i] = task_utility(task, worker)

    # Preference lists
    w_prefs = [np.argsort(-w_utils[i]) for i in range(len(workers))]
    t_prefs = [np.argsort(-t_utils[j]) for j in range(len(tasks))]

    # Initialize
    proposals = np.zeros(len(workers), dtype=int)
    worker_assignments = [[] for _ in range(len(workers))]
    task_assignments = [[] for _ in range(len(tasks))]
    unmatched = list(range(len(workers)))

    # Matching loop
    while unmatched:
        i = unmatched.pop(0)
        if proposals[i] >= len(tasks):
            continue

        j = w_prefs[i][proposals[i]]
        proposals[i] += 1

        if len(task_assignments[j]) < capacity:
            worker_assignments[i].append(j)
            task_assignments[j].append(i)
        else:
            # Find worst assigned worker
            worst_idx = min(task_assignments[j], key=lambda x: t_utils[j, x])
            if t_utils[j, i] > t_utils[j, worst_idx]:
                # Replace worker
                worker_assignments[worst_idx].remove(j)
                task_assignments[j].remove(worst_idx)
                unmatched.append(worst_idx)

                worker_assignments[i].append(j)
                task_assignments[j].append(i)
            else:
                unmatched.append(i)

    return worker_assignments, task_assignments

# 4.2 Classic Gale-Shapley
def classic_gs(workers, tasks, capacity=CAPACITY):
    # Static utilities
    w_utils = np.zeros((len(workers), len(tasks))
    t_utils = np.zeros((len(tasks), len(workers)))

    for i, worker in enumerate(workers):
        for j, task in enumerate(tasks):
            w_utils[i, j] = worker_utility(worker, task)
            t_utils[j, i] = task_utility(task, worker)

    # Same as RAGS but without dynamic adjustments
    return rags_matching(workers, tasks, capacity)

# 4.3 Random Allocation
def random_matching(workers, tasks, capacity=CAPACITY):
    worker_assignments = [[] for _ in range(len(workers))]
    task_assignments = [[] for _ in range(len(tasks))]

    # Shuffle all possible assignments
    all_pairs = [(i, j) for i in range(len(workers)) for j in range(len(tasks))]
    random.shuffle(all_pairs)

    for i, j in all_pairs:
        if (len(worker_assignments[i]) < capacity and
            len(task_assignments[j]) < capacity and
            np.all(workers[i].skill >= tasks[j].required_skill)):
            worker_assignments[i].append(j)
            task_assignments[j].append(i)

    return worker_assignments, task_assignments

# 4.4 FML (Federated Matching without RL)
def fml_matching(workers, tasks, capacity=CAPACITY):
    # Average weights across workers
    avg_weights = np.mean([w.weights for w in workers], axis=0)

    # Override worker preferences with global average
    for worker in workers:
        worker.weights = avg_weights.copy()

    return classic_gs(workers, tasks, capacity)

# 4.5 SenseChain+ Simulation
def sensechain_matching(workers, tasks, capacity=CAPACITY):
    # Simulate blockchain overhead
    time.sleep(0.001 * len(tasks))  # 1ms per task

    # Same as classic GS
    return classic_gs(workers, tasks, capacity)

############################
### 5. Evaluation Metrics ##
############################
def calculate_satisfaction(workers, tasks, assignments):
    satisfaction = []
    for i, worker in enumerate(workers):
        utils = [worker_utility(worker, tasks[j]) for j in assignments[i]]
        satisfaction.append(np.mean(utils) if utils else 0)
    return np.mean(satisfaction)

def calculate_task_coverage(task_assignments):
    return sum(len(a) > 0 for a in task_assignments) / len(task_assignments)

def calculate_quality(workers, tasks, assignments):
    # Simulate data submissions with noise
    qualities = []
    for i, worker in enumerate(workers):
        for task_id in assignments[i]:
            task = tasks[task_id]
            # Quality depends on worker skill and task difficulty
            base_quality = np.mean(worker.skill - task.required_skill) / 5
            noise = np.random.normal(0, 0.2 * (1 - worker.reputation))
            qualities.append(max(0, min(1, base_quality + noise)))
    return np.mean(qualities) if qualities else 0

def check_stability(workers, tasks, worker_assignments, task_assignments):
    blocking_pairs = 0
    sample_size = min(1000, len(workers))

    for i in np.random.choice(len(workers), sample_size, replace=False):
        worker = workers[i]
        current_utils = [worker_utility(worker, tasks[j]) for j in worker_assignments[i]]
        current_min = min(current_utils) if current_utils else -np.inf

        for j in np.random.choice(len(tasks), min(100, len(tasks)), replace=False):
            if j in worker_assignments[i]:
                continue

            # Worker prefers this task
            w_util = worker_utility(worker, tasks[j])
            if w_util <= current_min:
                continue

            # Task prefers this worker
            t_util = task_utility(tasks[j], worker)
            current_workers = task_assignments[j]
            if not current_workers:
                blocking_pairs += 1
                continue

            min_current_util = min(task_utility(tasks[j], workers[k]) for k in current_workers)
            if t_util > min_current_util:
                blocking_pairs += 1

    return 1 - (blocking_pairs / (sample_size * 100))

##########################
### 6. Ablation Models ###
##########################
def pota_no_rl(workers, tasks):
    """POTA without RL - use static weights"""
    return rags_matching(workers, tasks)

def pota_no_dqrs(workers, tasks):
    """POTA without DQRS - no reputation in utilities"""
    original_reputations = [w.reputation for w in workers]
    for w in workers:
        w.reputation = 0.8  # Default value

    assignments = rags_matching(workers, tasks)

    # Restore reputations
    for w, rep in zip(workers, original_reputations):
        w.reputation = rep

    return assignments

def pota_centralized(workers, tasks):
    """POTA without federated learning - centralized RL"""
    # Train a single global model
    global_model = LSTMPolicy().to(device)
    optimizer = optim.Adam(global_model.parameters(), lr=0.005)

    # Train on aggregated data
    states = torch.randn(10000, 1, 5, device=device)
    actions = torch.randint(0, 3, (10000,), device=device)
    rewards = torch.rand(10000, device=device)

    for epoch in range(20):
        outputs = global_model(states)
        loss = -torch.log(outputs[torch.arange(len(actions)), actions]) * rewards
        loss = loss.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Apply global weights
    with torch.no_grad():
        for worker in workers:
            state = torch.randn(1, 1, 5, device=device)
            new_weights = global_model(state).cpu().numpy()[0]
            worker.weights = new_weights

    return rags_matching(workers, tasks)

########################
### 7. Main Simulation #
########################
def run_full_simulation():
    print(f"🚀 Starting large-scale simulation with {N_WORKERS} workers and {N_TASKS} tasks")
    results = []
    methods = {
        "POTA (Proposed)": rags_matching,
        "Classic GS": classic_gs,
        "Random Allocation": random_matching,
        "FML": fml_matching,
        "SenseChain+": sensechain_matching,
        "POTA w/o RL": pota_no_rl,
        "POTA w/o DQRS": pota_no_dqrs,
        "POTA Centralized": pota_centralized
    }

    # Generate workers and tasks
    print("Generating workers and tasks...")
    workers = [Worker(i) for i in tqdm(range(N_WORKERS))]
    tasks = [Task(j) for j in tqdm(range(N_TASKS))]

    # Train DQRS model
    print("Training DQRS VAE...")
    vae_model = QualityVAE().to(device)
    optimizer = optim.Adam(vae_model.parameters(), lr=0.001)

    # Generate synthetic quality features
    features = torch.randn(100000, 3, device=device)
    for epoch in tqdm(range(50)):
        recon, mu, logvar = vae_model(features)
        loss = vae_loss(recon, features, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Federated RL Training
    print("Training federated RL agents...")
    worker_models = []
    batch_size = min(100, len(workers))

    for i in tqdm(range(0, len(workers), batch_size)):
        batch = workers[i:i+batch_size]
        batch_models = [train_worker_rl(w, tasks) for w in batch]
        worker_models.extend(batch_models)

        # Clear memory
        if GPU_ACCELERATED:
            torch.cuda.empty_cache()

    # Apply federated averaging
    print("Applying federated averaging...")
    global_weights = federated_average(worker_models)
    for i, model in enumerate(worker_models):
        model.load_state_dict(global_weights)
        workers[i].weights = model(torch.randn(1, 1, 5, device=device)).cpu().detach().numpy()[0]

    # Run all matching algorithms
    for method_name, matching_fn in methods.items():
        print(f"\n🔍 Running {method_name}...")
        start_time = time.time()

        # Reset assignments
        for w in workers:
            w.assigned_tasks = []

        # Run matching
        worker_assignments, task_assignments = matching_fn(workers, tasks)
        elapsed = time.time() - start_time

        # Calculate metrics
        satisfaction = calculate_satisfaction(workers, tasks, worker_assignments)
        coverage = calculate_task_coverage(task_assignments)
        quality = calculate_quality(workers, tasks, worker_assignments)
        stability = check_stability(workers, tasks, worker_assignments, task_assignments)

        # Record results
        results.append({
            "Method": method_name,
            "Satisfaction": satisfaction,
            "Task Coverage": coverage,
            "Data Quality": quality,
            "Stability": stability,
            "Time (s)": elapsed
        })

        print(f"  Satisfaction: {satisfaction:.3f} | Coverage: {coverage:.1%} | "
              f"Quality: {quality:.3f} | Stability: {stability:.1%} | Time: {elapsed:.1f}s")

    # Save results
    results_df = pd.DataFrame(results)
    results_df.to_csv("pota_results.csv", index=False)
    print("\n✅ Results saved to pota_results.csv")

    # Visualize results
    plot_results(results_df)

    return results_df

def plot_results(results_df):
    plt.figure(figsize=(14, 10))

    # Satisfaction and Quality
    plt.subplot(2, 2, 1)
    sns.barplot(x="Method", y="Satisfaction", data=results_df)
    plt.title("Worker Satisfaction")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)

    plt.subplot(2, 2, 2)
    sns.barplot(x="Method", y="Data Quality", data=results_df)
    plt.title("Data Quality Score")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)

    # Coverage and Stability
    plt.subplot(2, 2, 3)
    sns.barplot(x="Method", y="Task Coverage", data=results_df)
    plt.title("Task Coverage")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)

    plt.subplot(2, 2, 4)
    sns.barplot(x="Method", y="Stability", data=results_df)
    plt.title("Matching Stability")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig("pota_metrics.png")

    # Performance comparison
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x="Method", y="Time (s)", data=results_df)
    plt.title("Computation Time")
    plt.xticks(rotation=45)

    plt.subplot(1, 2, 2)
    efficiency = results_df["Satisfaction"] * results_df["Data Quality"] / results_df["Time (s)"]
    results_df["Efficiency"] = efficiency
    sns.barplot(x="Method", y="Efficiency", data=results_df)
    plt.title("Satisfaction-Quality/Time Efficiency")
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig("pota_performance.png")
    print("\n📊 Visualizations saved to pota_metrics.png and pota_performance.png")

# Run the full simulation
if __name__ == "__main__":
    results_df = run_full_simulation()

    # Display final results
    print("\n⭐ Final Results ⭐")
    print(results_df[['Method', 'Satisfaction', 'Data Quality', 'Stability', 'Time (s)']])